In [ ]:
import pandas
import os
import numpy
import plotly.express
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef, average_precision_score
from sklearn.tree import plot_tree
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import xgboost
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
df = pandas.read_csv("data/Network datatset/csv/attack_1.csv", encoding="us-ascii", delimiter=',', skipinitialspace=True)

In [ ]:
print(df)

In [ ]:
df.head()

In [ ]:
print(df.keys())

In [ ]:
nb_attaque= df['label'].value_counts()
print(nb_attaque)
ratio = (nb_attaque.iloc[1]+nb_attaque.iloc[2]+nb_attaque.iloc[3])/(nb_attaque.iloc[0]+nb_attaque.iloc[1]+nb_attaque.iloc[2]+nb_attaque.iloc[3])
print(ratio*100)

In [ ]:
print(nb_attaque)

In [ ]:
df['modbus_fn'].value_counts()

In [ ]:
colonnes_catégorielles = df.select_dtypes(include=['category', 'object']).columns.tolist()

colonnes_numériques = df.select_dtypes(include=['number']).columns.tolist()

print("Colonnes catégorielles :")
print(colonnes_catégorielles)

print("\nColonnes numériques :")
print(colonnes_numériques)

In [ ]:
cat_stats = df[colonnes_catégorielles].describe()
print(cat_stats)
print()
valeur_catégorielles = {}
for col in colonnes_catégorielles:
    valeur_catégorielles[col] = df[col].unique()
    print("%s: %s" % (col, valeur_catégorielles[col]))

In [ ]:
numeric_stats = df[colonnes_numériques].describe()

print("Numeric Columns:")
print(colonnes_numériques)

print("\nStatistics for Numeric Columns:")
print(numeric_stats)

In [ ]:
print(df.head())
print(df['label'].value_counts())

for v in ('Time', 'mac_s', 'mac_d', 'ip_s', 'ip_d', 'proto', 'modbus_fn', 'modbus_response', 'label'):
    print("Number of unique values for {}: {}".format(v, df[v].nunique()))

In [ ]:
nan_ratios = df.isna().mean()   
print(nan_ratios)


In [ ]:
import pandas as pd


df_filtre = df[df['modbus_fn'].isna()]

nombre_de_1 = df_filtre['label_n'].sum()
print("Le nombre de 1 dans la colonne 'label' pour les valeurs manquantes dans 'modbus_fn' est :", nombre_de_1)
print(df_filtre.shape)


In [ ]:
target_attack_categories  = df['label'].unique()
target_attack_categories = target_attack_categories[1:4]
print(target_attack_categories)

In [ ]:
# on récupère tous les labels 1
MM = df[(df['label_n'] == 1)]
#on récupère toute les colonnes où il y a bien une attaque pour un label 1
matching_rows = (MM['label'].isin(target_attack_categories))
# on vérifie que l'on a bien que la valeur True
print(matching_rows.all())

In [ ]:
# on récupère tous les labels 0
MM = df[(df['label_n'] == 0)]

#on récupère toute les colonnes où il y a bien une attaque pour un Label 0
matching_rows =(MM['label'].isin(["normal"]))
# on vérifie que l'on a bien que la valeur True
print(matching_rows.all())

In [ ]:
nb_fraud = df['label_n'].value_counts()
print(nb_fraud)
ratio = nb_fraud[1]/(nb_fraud[0]+nb_fraud[1])
print(ratio*100)

In [ ]:
# Create a new DataFrame X you will use for cleaning.
fraudulent_entries = df[df['label_n'] == 1].sample(n=int(10000 * ratio) +1)

# Sample non-fraudulent entries to match the ratio
non_fraudulent_entries = df[df['label_n'] == 0].sample(n=int(10000 * (1-ratio)))

# Concatenate both sets to create df_reduced
X = pandas.concat([fraudulent_entries, non_fraudulent_entries])
print(X.shape)


In [ ]:

# Remplacez les valeurs '[int]' par 'int' et transformez la colonne en numérique
X['modbus_response'] = X['modbus_response'].str.replace(r'\[(\d+)\]', r'\1', regex=True)
X['modbus_response'] = pandas.to_numeric(X['modbus_response'], errors='coerce')
X['modbus_response'] = X['modbus_response'].fillna(-1)

In [ ]:
X = X.dropna()


In [ ]:
nan_ratios = X.isna().mean()   
print(nan_ratios)

In [ ]:
colonnes_catégorielles = X.select_dtypes(include=['category', 'object']).columns.tolist()

# Extraction des noms des colonnes numériques
colonnes_numériques = X.select_dtypes(include=['number']).columns.tolist()

print("Colonnes catégorielles :")
print(colonnes_catégorielles)

print("\nColonnes numériques :")
print(colonnes_numériques)

In [ ]:
# Store the attack_cat labels in a DataFrame Y as a 1-D array
Y = X['label_n'].copy()

In [ ]:
print(Y)

In [ ]:
X.drop(['label', 'Time','label_n'], axis=1, inplace=True)

In [ ]:
X = pandas.get_dummies(X)

In [ ]:
# Extract train and test data through train_test_split on X and Y
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_proba = clf.predict_proba(X_test)
print(Y_pred)

In [ ]:
print('Confusion matrix:', confusion_matrix(Y_test, Y_pred))
print('Precision:', precision_score(Y_test, Y_pred))
print('Recall:', recall_score(Y_test, Y_pred))
print('Accuracy:', accuracy_score(Y_test, Y_pred))
print('Balanced accuracy:', balanced_accuracy_score(Y_test, Y_pred))
print('Matthews Correlation Coefficient:', matthews_corrcoef(Y_test, Y_pred))
print('AUPRC:', average_precision_score(Y_test, Y_pred))
fig = plotly.express.imshow(confusion_matrix(Y_test, Y_pred), title="Confusion matrix")
fig.show()

In [ ]:
# For the XGBClassifier interface in sklearn
# 1. Perform the same analysis
# 2. Plot the relative importance of fields through plot_importance, using importance_type ‘cover’, ‘gain’, ‘weight’
# 3. Show the classifier visually

'''
clf = xgboost.XGBClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_proba = clf.predict_proba(X_test)

print('Confusion matrix:', confusion_matrix(Y_test, Y_pred))
print('Precision:', precision_score(Y_test, Y_pred))
print('Recall:', recall_score(Y_test, Y_pred))
print('Accuracy:', accuracy_score(Y_test, Y_pred))
print('Balanced accuracy:', balanced_accuracy_score(Y_test, Y_pred))
print('Matthews Correlation Coefficient:', matthews_corrcoef(Y_test, Y_pred))
print('AUPRC:', average_precision_score(Y_test, Y_pred))

fig = plotly.express.imshow(confusion_matrix(Y_test, Y_pred), title='Confusion matrix')
fig.show()
'''

In [ ]:
'''

# Liste des valeurs de k à tester
k_values = range(1, 10)

# Liste pour stocker les scores de précision
accuracy_scores = []
recall_scores = []
# Test de chaque valeur de k
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(Y_test, y_pred)
    recall = recall_score(Y_test, y_pred)
    accuracy_scores.append(accuracy)
    recall_scores.append(recall)

# Création d'un DataFrame pour utiliser plotly.express
data = {'k': k_values, 'Précision': accuracy_scores, 'Rappel': recall_scores}
D = pd.DataFrame(data)

# Tracer la courbe du critère du coude avec plotly.express
fig = plotly.express.line(D, x='k', y=['Précision', 'Rappel'], markers=True,
              title='Critère du Coude pour le KNN')
fig.update_layout(xaxis_title='Nombre de voisins (k)', yaxis_title='Score')
fig.show()

#Best precision for k=1
'''


In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
Y_pred_proba = knn.predict_proba(X_test)

print("Confusion matrix:", confusion_matrix(Y_test, Y_pred))
print("Precision:", precision_score(Y_test, Y_pred))
print("Recall:", recall_score(Y_test, Y_pred))
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Balanced accuracy:", balanced_accuracy_score(Y_test, Y_pred))
print("Matthews Correlation Coefficient:", matthews_corrcoef(Y_test, Y_pred))
print("AUPRC:", average_precision_score(Y_test, Y_pred))

fig = plotly.express.imshow(confusion_matrix(Y_test, Y_pred), title="Confusion matrix")
fig.show()